<a href="https://colab.research.google.com/github/ZorislavVyymov/NEW_3/blob/main/%D0%98%D1%81%D1%81%D0%BB%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_DS_%D0%9A%D0%BB%D0%B0%D1%81%D1%82%D0%B5%D1%80%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F_%D0%BE%D1%82%D0%B2%D0%B5%D1%82%D0%BE%D0%B2_%D0%A1%D0%A2%D0%9F_%D0%94%D0%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Цели/задачи исследования**

**Тема**: Исследование базы данных обращений в СТП по направлению ДО (модули досудебного обжалования) с использованием методов кластеризации.

**Зачем мы это делаем**: параллельно с изучением темы кластеризации, на практике изучить возможность автоматической группировки похожих ответов на обращения в СТП с проверкой гипотезы: примерно одинаковые ответы/решения даны на похожие вопросы/обращения, а значит группировку похожих обращений можно использовать как БД для "узнавания" проблемы с которой обратился пользователь которая дл группы решается примерно одинаково.

**Тактика**: Сначала тексты фактических ответов превращаем в числовые векторы, затем делим их на группы (кластеры) с помощью алгоритмов, а важные фрагменты объединяем и структурируем для удобного использования с помощью современных технологий, таких как FAISS и GPT-модели.

**Цели**:
1. Закрепление на практике темы курса
2. Освоить методы кластеризации данных, такие как K-Means и DBSCAN.
3. Опробовать способы снижения размерности данных с использованием PCA, t-SNE и UMAP для визуальной аналитики полученных кластеров
4. Научиться очищать данные от дубликов и схожих записей с помощью метода TF-IDF на правктических данных
5. Проверить гипотезу.
6. Создать и использовать учебную векторную базу данных с применением FAISS как прототип боевой, если эффективность методов для целей проекта будет подтверждена.
6. Научиться объединять важные текстовые чанки в единый документ кластера (**гигачанк**) с использованием фильтрации и структурирования на основе GPT-моделей.


Мспользуем методы кластеризации ответов по обращениям для автоматического объединения содержания самих обращений в группы обращений с одинаковыми (похожими) проблемами.  Возможно (проверяем гипотезу), это окажется более точным методом классификации обращений и подбора ответа/способа решения

Выделить обращения, которые выбиваются из общей массы


# Обработка Решений СТП по ДО

### Подготовка. Установка библиотек, подключение к OpenAI...

In [1]:
# @title Установка библиотек
!pip install -q umap-learn==0.5.7
!pip install -q langchain langchain-community langchain-core langchain-openai langchain-text-splitters langsmith openai faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [10]:
# @title Импорты

import os
import copy
import re
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import json
from tqdm import tqdm_notebook as tqdm # используется для отображения загрузки (красивого)
from sklearn.cluster import KMeans     # для кластеризации
from sklearn.decomposition import PCA  # для синижени размерности отображаемых векторов (сами вектора не изменяются)
from google.colab import userdata       # для использования пользовательских переменных в колабе (например ключей доступа в OAI)
from langchain.text_splitter import (   # импорт нескольких сплиттеров
    CharacterTextSplitter,
    NLTKTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain.docstore.document import Document    # служба angchain для обработчиков документов
from langchain.document_loaders import TextLoader   # служба angchain для обработчиков загрузчика текста
from langchain_openai import OpenAIEmbeddings       # класс для работы с эмбедингами
from langchain_community.vectorstores import FAISS  # загрузка БД FAISS  - класс для работы с векторными хранилищами данных
import openai
from openai import AsyncOpenAI, OpenAI              # две реализации библиотеки OpenAI (асинхронная и синхронная)
import tiktoken                                     # для подсчета токенов и затрат на обращение в OAI

warnings.filterwarnings("ignore")                   # это команда, которая заставляет Python молчать о предупреждениях!

In [2]:
from google.colab import drive                      # монтируем гугл-диск к колабу
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title Функция для загрузки данных из Яндекс.Диска в рабочий каталог **root_path**

import os
import requests
import zipfile
import shutil

def download_and_extract_yandex_disk(url, output_dir):
    """
    Скачивает архив с Яндекс.Диска и распаковывает содержимое корневой папки в указанный каталог.

    :param url: Ссылка на файл или папку на Яндекс.Диске (публичная).
    :param output_dir: Папка, куда будет распаковано содержимое архива.
    """
    os.makedirs(output_dir, exist_ok=True)
    zip_file_path = os.path.join(output_dir, 'data.zip')

    # Преобразование ссылки для прямого скачивания
    direct_url = f"https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key={url}"
    response = requests.get(direct_url)
    if response.status_code != 200:
        print(f"Ошибка при запросе ссылки: {response.status_code}")
        return False

    download_link = response.json().get('href')
    if not download_link:
        print("Не удалось получить ссылку для скачивания.")
        return False

    # Скачиваем файл
    with requests.get(download_link, stream=True) as file_response:
        with open(zip_file_path, 'wb') as file:
            shutil.copyfileobj(file_response.raw, file)
    print(f"Файл успешно загружен: {zip_file_path}")

    # Распаковываем содержимое архива
    if zipfile.is_zipfile(zip_file_path):
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            for member in zip_ref.namelist():
                # Извлекаем только файлы без корневой папки
                member_path = os.path.relpath(member, start=os.path.commonpath(zip_ref.namelist()))
                target_path = os.path.join(output_dir, member_path)
                if not member.endswith('/'):  # Пропускаем директории
                    os.makedirs(os.path.dirname(target_path), exist_ok=True)
                    with open(target_path, 'wb') as f:
                        f.write(zip_ref.read(member))
        print(f"Файлы успешно распакованы в папку: {output_dir}")
        os.remove(zip_file_path)  # Удаляем архив после распаковки
        return True
    else:
        print("Скачанный файл не является ZIP-архивом.")
        return False

In [11]:
# @title Служебные функции и переменные ноутбука

# Подсчет токенов в строке
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    return len(encoding.encode(string))

# Функция для расчета стоимости запросов
def calculate_cost(complition_questions, usd_price, verbose=False):
    """
    Функция для расчета стоимости входных и выходных токенов, а также общей стоимости запроса.

    :param complition_questions: Объект с информацией о токенах (предполагается, что он имеет атрибуты usage.prompt_tokens и usage.completion_tokens)
    :param usd_price: Цена одного доллара в рублях
    :return: Словарь с рассчитанной стоимостью
    """
    # print('complition_questions ', complition_questions)

    input_price = pricing_per_million_tokens_usd.get(complition_questions.model)["input_price"]
    output_price = pricing_per_million_tokens_usd.get(complition_questions.model)["output_price"]

    prompt_tokens_cost = ((int(complition_questions.usage.prompt_tokens) * input_price) / 1000000) * usd_price
    completion_tokens_cost = ((int(complition_questions.usage.completion_tokens) * output_price) / 1000000) * usd_price
    total_cost = prompt_tokens_cost + completion_tokens_cost

    cost_details = {
        "prompt_tokens_cost": prompt_tokens_cost,
        "completion_tokens_cost": completion_tokens_cost,
        "total_cost": total_cost
    }
    if verbose:
        print(f"Цена {complition_questions.usage.prompt_tokens} входных токенов, руб: {prompt_tokens_cost}")
        print(f"\nЦена {complition_questions.usage.completion_tokens} выходных токенов, руб: {completion_tokens_cost}")
        print(f"\nЦена запроса, руб: {total_cost}")

    return cost_details

def calculate_total_cost(data):
    # Инициализируем переменную для общей стоимости
    total_cost = 0

    # Проходим по каждому элементу списка и извлекаем значение 'total_cost'
    for item in data:
        for key, value in item.items():
            total_cost += value['total_cost']

    # Печатаем общую стоимость
    return f"Общая стоимость вызовов для запроса: {total_cost:.5f} руб"


# Корневая папка проекта ВСХ
root_path = '/content/drive/MyDrive/Обучение ВСХ/Проект ВСХ/'

# Пути и другие переменные ноутбука
db_index_path = "/faiss_index/"
text_file = 'base_uii_edit_22.txt'

# Модель GPT
model_answer = 'gpt-4o-mini'

len_chunk = 1024


# Производим загрузку данных для чекпоинтов с Яндекс.Диска
#files_url = "https://disk.yandex.ru/d/94pepH9co6ZijQ"
#local_path = '/content/local_data'

#if download_and_extract_yandex_disk(files_url, local_path):
    # Переназначаем корневую папку занятия на папку загрузки с Яндекс.Диска
#    root_path = local_path + '/'
#    print(f'Корневая папка занятия изменена на: {root_path}')

In [12]:
# @title Установка цен для расчета стоимости запросов
usd_price = 82                 # устанавливаем текущий курс доллара USA
encoding_name = "cl100k_base"  # Это имя кодировки, в которой сохранена наша векторная БД
model_parsed = 'gpt-4o-mini-2024-07-18'
# Define pricing embed for each model in USD per 1M tokens
pricing_per_million_tokens_usd = {                                  # если текущий тариф отличается, параметры надо скорректировать (автообновление пока невозможно)
    "gpt-4o-mini": {"input_price": 0.15, "output_price": 0.6},
    "gpt-4o-mini-2024-07-18": {"input_price": 0.15, "output_price": 0.6},
    "gpt-4o": {"input_price": 2.5, "output_price": 10},
    "gpt-4o-2024-08-06": {"input_price": 2.5, "output_price": 10},
    "gpt-4o-2024-05-13": {"input_price": 10, "output_price": 15},
    "text-embedding-3-small": 0.020,
    "text-embedding-3-large": 0.130,
    "text-embedding-ada-002": 0.100
    }

history_cost = []

In [13]:
# @title Загружаем ключ OpenAI

openai.api_key = userdata.get("OPENAI_API_KEY")   # Из переменной окружения загружам ключ OAI (проверить совпадение название)
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

model_embed_large = "text-embedding-3-large"      # Указываем модель для эмбедингов (большая)
embeddings_large = OpenAIEmbeddings(model=model_embed_large)

model_embed = "text-embedding-ada-002"  # модель для эмбединга поменьше: ["text-embedding-3-small", "text-embedding-3-large", "text-embedding-ada-002"]
embeddings = OpenAIEmbeddings(model=model_embed)

### Загрузка данных и получение их векторного представления (эмбеддингов)

In [14]:
# @title Загрузка документа с Ответами СТП

def load_unique_answers (file_path):
    # Чтение файла xlsx
    df = pd.read_excel(file_path)

    # Извлечение списка ответов СТП
    unique_answers = df['Решение'].tolist()                    # указываем из какого столбца (название) извлекать ответы в список

    return unique_answers

# Корневая папка проекта ВСХ
root_path = '/content/drive/MyDrive/Обучение ВСХ/Проект ВСХ/'

file_path = root_path + 'Список решений по обращениям.xlsx'   # указываем путь к xlsx-документу с ответами СТП
unique_answers = load_unique_answers (file_path)

In [15]:
# @title
unique_answers

['Добрый день! Ошибка на дашборде не подтверждена. Просьба подтвердите\nактуальность проблемы в ответном письме. К письму необходимо приложить\nскриншоты и подробное описание текущего состояния проблемы',
 'Добрый день! На данный момент виды контроля выводятся из ЕРВК, справочники ЕС\nНСИ больше не сопровождаются.',
 'Добрый день! Указанный дашборд не сопровождается разработчиками, актуальные\nданные представлены в ГАС У.',
 'Добрый день! Указанный дашборд не сопровождается разработчиками, актуальные\nданные представлены в ГАС У.',
 'Добрый день! Указанные уведомления приходят пользователям с ролью "Помощник\nруководителя"',
 'Добрый день!\nОтчет доработан разработчиком.\n\nС уважением, \nСТП ГИС ТОР КНД',
 'Добрый день!\nУказанный дашборд фактически не сопровождается разработчиками.\nНа сопровождение СТП этот инструмент не передавался, эксплуатационная\nдокументация по нему отсутствует. \nПредлагаем не использовать указанный дашборд, т.к. корректность данных в не\nобеспечивается.\n\nС

In [ ]:
# сколько всего вопросов в списке?
len(unique_answers)

1036

In [ ]:
# @title Получение векторного представления ответов СТП (эмбеддингов) <br />**OpenAIEmbeddings(model="text-embedding-3-large")**
# Получаем векторные представления ответов СТП
vectors_answ = embeddings_large.embed_documents(unique_answers)

# Преобразуем векторы в numpy массив
vectors_answ = np.array(vectors_answ)

In [ ]:
vectors_answ

array([[-0.01692186,  0.00122831,  0.00966179, ..., -0.02086069,
         0.01329869,  0.00465934],
       [ 0.01579257, -0.00485329, -0.00355896, ..., -0.00834006,
         0.01641244, -0.00197055],
       [-0.02406687, -0.00754398,  0.00361571, ..., -0.00874448,
         0.00776419, -0.00424083],
       ...,
       [-0.02586235, -0.03263133,  0.00891294, ..., -0.02358355,
         0.0127424 ,  0.01179178],
       [-0.00584543, -0.03182739,  0.00992449, ...,  0.0025222 ,
         0.03271468,  0.00211553],
       [-0.01120884, -0.01986153,  0.00738197, ...,  0.00241658,
         0.01220779, -0.00294361]])

In [ ]:
# какой длины одна [первая] фраза?
len(vectors_answ[0])

3072

In [ ]:
# @title Вывод 5 первых строк из БД и их векторного представления

print('----------------------------------------------------------------')
print(f'\033[1;035mПервые 5 вопросов из базы\033[0m (\033[1;092m{len(unique_answers)}\033[0m строк)')
display(unique_answers[:5])
print(f'\n\033[1;034mИх векторное представление\033[0m (\033[1;091m{len(vectors_answ)}\033[0m векторов)')
display(vectors_answ[:5])
print('----------------------------------------------------------------')

----------------------------------------------------------------
Первые 5 вопросов из базы (1036 строк)


['Добрый день! Ошибка на дашборде не подтверждена. Просьба подтвердите\nактуальность проблемы в ответном письме. К письму необходимо приложить\nскриншоты и подробное описание текущего состояния проблемы',
 'Добрый день! На данный момент виды контроля выводятся из ЕРВК, справочники ЕС\nНСИ больше не сопровождаются.',
 'Добрый день! Указанный дашборд не сопровождается разработчиками, актуальные\nданные представлены в ГАС У.',
 'Добрый день! Указанный дашборд не сопровождается разработчиками, актуальные\nданные представлены в ГАС У.',
 'Добрый день! Указанные уведомления приходят пользователям с ролью "Помощник\nруководителя"']


Их векторное представление (1036 векторов)


array([[-0.01692186,  0.00122831,  0.00966179, ..., -0.02086069,
         0.01329869,  0.00465934],
       [ 0.01579257, -0.00485329, -0.00355896, ..., -0.00834006,
         0.01641244, -0.00197055],
       [-0.02406687, -0.00754398,  0.00361571, ..., -0.00874448,
         0.00776419, -0.00424083],
       [-0.02406687, -0.00754398,  0.00361571, ..., -0.00874448,
         0.00776419, -0.00424083],
       [-0.02376362, -0.05350949,  0.01243317, ..., -0.0055584 ,
         0.01046895, -0.03743735]])

----------------------------------------------------------------


**Внимание!** Результаты векторизации vectors_answ могут отличаться при повторном запуске

In [ ]:
# @title Сохраняем Ответы СТП и результат их векторизации (**unique_answers** и **vectors**)

# Сохранение векторов в файл
with open(root_path + 'vectors_answ.npy', 'wb') as f:
     np.save(f, vectors_answ)
     print('Векторы успешно сохранены в vectors_answ.npy')     #

unique_answers_json = json.dumps(unique_answers, ensure_ascii=False)
with open(root_path + 'unique_answers.json', 'w', encoding='utf-8') as f:
     f.write(unique_answers_json)
     print('Список ответов СТП успешно сохранён в unique_answers.json')

Векторы успешно сохранены в vectors_answ.npy
Список ответов СТП успешно сохранён в unique_answers.json


In [16]:
# @title Чекпоинт (загрузка **unique_answers** и **vectors_answ**) <br />[Следующий чекпоинт](#scrollTo=2woh3rIxXaiR)

with open(root_path + 'vectors_answ.npy', 'rb') as f:
    vectors_answ = np.load(f)
    print('Векторы успешно загружены из vectors_answ.npy')
with open(root_path + 'unique_answers.json', 'r', encoding='utf-8') as f:
    unique_questions = json.load(f)
    print('Список вопросов успешно загружен из unique_answers.json')

Векторы успешно загружены из vectors_answ.npy
Список вопросов успешно загружен из unique_answers.json


---
---
---

## Попробуем найти крупные кластеры методом **DBSCAN** (для сравнения методов кластеризации)

In [17]:
# @title Кластеризация методом DBSCAN. Анализ состава кластеров с текстами
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_distances

# Загрузка сохраненных данных
with open(root_path + 'vectors_answ.npy', 'rb') as f:
    vectors = np.load(f)
    print(f'Загружены векторы размерности: {vectors.shape}')

with open(root_path + 'unique_answers.json', 'r', encoding='utf-8') as f:
    unique_answers = json.load(f)
    print(f'Загружено текстов: {len(unique_answers)}')

# Проверяем соответствие размеров
print(f"Количество векторов: {len(vectors)}")
print(f"Количество текстов: {len(unique_answers)}")

# Выполняем кластеризацию DBSCAN (ваш исходный код)
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_distances

# Параметры кластеризации
eps = 0.16              # Радиус окрестности    # данный DS в диапазоне 0.28 - 0.1 дает макс 4 кластера  (0.16)
min_samples = 25          # Минимальное количество точек для формирования кластера
metric = 'precomputed'    # Метрика расстояния

# Вычисление косинусной матрицы расстояний
distance_matrix = cosine_distances(vectors)

# Выполнение кластеризации DBSCAN
dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric=metric)
clusters_dbscan = dbscan.fit_predict(distance_matrix)

# Создаем основной DataFrame с текстами и кластерами
results_df = pd.DataFrame({
    'Text': unique_answers,
    'Cluster': clusters_dbscan,
    'Text_Length': [len(str(text)) for text in unique_answers]
})

print("\n" + "="*60)
print("ОСНОВНАЯ СВОДКА ПО КЛАСТЕРАМ")
print("="*60)

# Анализируем результаты кластеризации
unique_labels = np.unique(clusters_dbscan)
num_clusters = len(unique_labels[unique_labels != -1])
num_noise = np.sum(clusters_dbscan == -1)

print(f"Всего кластеров: {num_clusters}")
print(f"Точек шума: {num_noise}")
print(f"Всего обработано текстов: {len(unique_answers)}")

# Выводим статистику по кластерам
print("\nРАСПРЕДЕЛЕНИЕ ТЕКСТОВ ПО КЛАСТЕРАМ:")
cluster_stats = results_df['Cluster'].value_counts().sort_index()
for cluster_id, count in cluster_stats.items():
    cluster_type = "🔴 ШУМ" if cluster_id == -1 else f"🔵 Кластер {cluster_id}"
    print(f"{cluster_type}: {count} текстов")

Загружены векторы размерности: (1036, 3072)
Загружено текстов: 1036
Количество векторов: 1036
Количество текстов: 1036

ОСНОВНАЯ СВОДКА ПО КЛАСТЕРАМ
Всего кластеров: 4
Точек шума: 800
Всего обработано текстов: 1036

РАСПРЕДЕЛЕНИЕ ТЕКСТОВ ПО КЛАСТЕРАМ:
🔴 ШУМ: 800 текстов
🔵 Кластер 0: 62 текстов
🔵 Кластер 1: 27 текстов
🔵 Кластер 2: 37 текстов
🔵 Кластер 3: 110 текстов


In [18]:
# @title Детальный просмотр текстов по кластерам

def view_cluster_details(cluster_id, max_texts=10):
    """Просмотр детального состава кластера"""
    cluster_data = results_df[results_df['Cluster'] == cluster_id]

    print(f"\n{'='*80}")
    if cluster_id == -1:
        print(f"🔴 ШУМ - {len(cluster_data)} текстов")
    else:
        print(f"🔵 КЛАСТЕР {cluster_id} - {len(cluster_data)} текстов")
    print(f"{'='*80}")

    # Показываем тексты
    for i, (idx, row) in enumerate(cluster_data.head(max_texts).iterrows(), 1):
        print(f"\n📝 Текст {i} (длина: {row['Text_Length']} символов):")
        print(f"   {row['Text']}")

    if len(cluster_data) > max_texts:
        print(f"\n... и ещё {len(cluster_data) - max_texts} текстов")

# Просмотр всех кластеров
for cluster_id in sorted(np.unique(clusters_dbscan)):
    view_cluster_details(cluster_id)


🔴 ШУМ - 800 текстов

📝 Текст 1 (длина: 198 символов):
   Добрый день! Ошибка на дашборде не подтверждена. Просьба подтвердите
актуальность проблемы в ответном письме. К письму необходимо приложить
скриншоты и подробное описание текущего состояния проблемы

📝 Текст 2 (длина: 107 символов):
   Добрый день! На данный момент виды контроля выводятся из ЕРВК, справочники ЕС
НСИ больше не сопровождаются.

📝 Текст 3 (длина: 104 символов):
   Добрый день! Указанный дашборд не сопровождается разработчиками, актуальные
данные представлены в ГАС У.

📝 Текст 4 (длина: 104 символов):
   Добрый день! Указанный дашборд не сопровождается разработчиками, актуальные
данные представлены в ГАС У.

📝 Текст 5 (длина: 89 символов):
   Добрый день! Указанные уведомления приходят пользователям с ролью "Помощник
руководителя"

📝 Текст 6 (длина: 74 символов):
   Добрый день!
Отчет доработан разработчиком.

С уважением, 
СТП ГИС ТОР КНД

📝 Текст 7 (длина: 303 символов):
   Добрый день!
Указанный дашборд фактическ

In [19]:
# @title Сохранение результатов кластеризации в файлы

# Сохраняем полную таблицу в CSV
results_df.to_csv(root_path + 'clustering_results.csv', index=False, encoding='utf-8-sig')
print("✅ Полные результаты сохранены в 'clustering_results.csv'")

# Сохраняем в Excel с группировкой по кластерам
with pd.ExcelWriter(root_path + 'detailed_clusters.xlsx') as writer:
    # Общая таблица
    results_df.to_excel(writer, sheet_name='All_Clusters', index=False)

    # Отдельные листы для каждого кластера
    for cluster_id in np.unique(clusters_dbscan):
        cluster_data = results_df[results_df['Cluster'] == cluster_id]
        sheet_name = f'Шум' if cluster_id == -1 else f'Кластер_{cluster_id}'
        sheet_name = sheet_name[:31]  # Ограничение Excel
        cluster_data.to_excel(writer, sheet_name=sheet_name, index=False)

print("✅ Детальные результаты по кластерам сохранены в 'detailed_clusters.xlsx'")

# Сохраняем сводную статистику
summary_stats = results_df.groupby('Cluster').agg({
    'Text': 'count',
    'Text_Length': ['mean', 'min', 'max']
}).round(2)

summary_stats.columns = ['Количество_текстов', 'Ср_длина_текста', 'Мин_длина', 'Макс_длина']
summary_stats.to_csv(root_path + 'clustering_summary.csv', encoding='utf-8-sig')
print("✅ Статистика по кластерам сохранена в 'clustering_summary.csv'")

✅ Полные результаты сохранены в 'clustering_results.csv'
✅ Детальные результаты по кластерам сохранены в 'detailed_clusters.xlsx'
✅ Статистика по кластерам сохранена в 'clustering_summary.csv'


In [20]:
# @title Анализ самых крупных кластеров

# Исключаем шум из анализа
clusters_without_noise = results_df[results_df['Cluster'] != -1]

# Находим топ-5 самых крупных кластеров
top_clusters = clusters_without_noise['Cluster'].value_counts().head(5)

print("🏆 ТОП-5 САМЫХ КРУПНЫХ КЛАСТЕРОВ:")
print("="*60)

for cluster_id, count in top_clusters.items():
    cluster_texts = results_df[results_df['Cluster'] == cluster_id]['Text'].tolist()

    print(f"\n🎯 Кластер {cluster_id} ({count} текстов)")
    print("Примеры текстов:")

    # Показываем разнообразные примеры
    sample_size = min(3, len(cluster_texts))
    for i in range(sample_size):
        text = cluster_texts[i]
        preview = str(text)[:120] + "..." if len(str(text)) > 120 else str(text)
        print(f"   • {preview}")

🏆 ТОП-5 САМЫХ КРУПНЫХ КЛАСТЕРОВ:

🎯 Кластер 3 (110 текстов)
Примеры текстов:
   • Добрый день!
Указанная Вами ошибка связана с устаревшей версией используемого подписантом
плагина.
Рекомендуем обновить ...
   • Добрый день!
Указанная Вами ошибка связана с устаревшей версией используемого подписантом
плагина.
Рекомендуем обновить ...
   • Добрый день!
Указанная Вами ошибка связана с устаревшей версией используемого подписантом
плагина.
Рекомендуем обновить ...

🎯 Кластер 0 (62 текстов)
Примеры текстов:
   • Добрый день!
Видим, что решение успешно удалось подписать и направить на портал
Завершаем данное обращение.
С уважением,...
   • Добрый день!
Работы по данному обращению завершены.
Решение предоставлено в рамках Вашего параллельного обращения 127096...
   • Добрый день!
Видим, что по обеим жалобам решение успешно подписано и направлено на портал.
Завершаем данное обращение, к...

🎯 Кластер 2 (37 текстов)
Примеры текстов:
   • Добрый день!

По Вашему обращению проведены работы, в рамк

**Визуализация:**

In [21]:
# @title Подготовка **t-SNE** для снижения размерности
from sklearn.manifold import TSNE

# Параметры для t-SNE
n_components = 2  # Снижение размерности до 2D
random_state = 42

# Применение t-SNE
tsne = TSNE(n_components=n_components, random_state=random_state, metric='precomputed', init='random')
tsne_results = tsne.fit_transform(distance_matrix)

# Создаём DataFrame с результатами t-SNE
tsne_df = pd.DataFrame(tsne_results, columns=['TSNE1', 'TSNE2'])
tsne_df['Cluster'] = clusters_dbscan

In [22]:
# Уникальные метки кластеров (включая шум)
unique_labels = np.unique(clusters_dbscan)
print(f"Все уникальные метки: {unique_labels}")

# Кластеры - это все метки, кроме -1
clusters = unique_labels[unique_labels != -1]
num_clusters = len(clusters)

print(f"Количество кластеров (без учета шума): {num_clusters}")
print(f"Количество точек, помеченных как шум: {np.sum(clusters_dbscan == -1)}")

Все уникальные метки: [-1  0  1  2  3]
Количество кластеров (без учета шума): 4
Количество точек, помеченных как шум: 800


In [25]:
# @title Визуализация кластеризации методом **DBSCAN** / **t-SNE**
import plotly.express as px
import plotly.graph_objects as go
from scipy.spatial import ConvexHull

# Добавляем столбец с вопросами в DataFrame для визуализации
tsne_df['ответы СТП'] = unique_answers

# Создаём scatter plot для кластеров
fig = px.scatter(
    tsne_df,
    x='TSNE1',
    y='TSNE2',
    color='Cluster',
    hover_data={'TSNE1': False, 'TSNE2': False, 'Cluster': True, 'ответы СТП': True},
    title='DBSCAN Clustering (t-SNE Visualization with Annotations)',
    color_discrete_sequence=px.colors.qualitative.Bold + px.colors.qualitative.Pastel
)

# Добавляем контуры кластеров
for cluster_id in tsne_df['Cluster'].unique():
    if cluster_id == -1:  # Пропускаем шумовые точки
        continue
    cluster_points = tsne_df[tsne_df['Cluster'] == cluster_id][['TSNE1', 'TSNE2']].values
    if len(cluster_points) >= 3:  # ConvexHull требует минимум 3 точки
        hull = ConvexHull(cluster_points)
        hull_points = cluster_points[hull.vertices]
        hull_points = np.append(hull_points, [hull_points[0]], axis=0)  # Замыкаем контур

        # Добавляем область кластера
        fig.add_trace(go.Scatter(
            x=hull_points[:, 0],
            y=hull_points[:, 1],
            mode='lines',
            line=dict(color='rgba(0,0,0,0.5)', width=1),
            fill='toself',
            fillcolor='rgba(0, 0, 0, 0.1)',
            name=f'Cluster {cluster_id} Area',
            visible='legendonly'  # По умолчанию скрыть области
        ))

# Настраиваем макет
fig.update_layout(
    height=800,
    coloraxis_colorbar=dict(title='Cluster', x=0.95),
    legend=dict(title='Clusters', itemsizing='constant')
)

fig.show()

## Кластеризация вопросов **K-Means**